In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install neattext

     |████████████████████████████████| 112kB 7.9MB/s 


In [ ]:
import pandas as pd
import string #Opérations courantes sur les chaînes
import re   # pour utiliser  des expressions régulières ex:$
import nltk #boites outils ntk
from sklearn.model_selection import train_test_split
import neattext as nt
import neattext.functions as nfx

In [ ]:
#Cleaning & Preprocessing
def process(text):
# la normalisation
    regex = re.compile(r'[إأٱآا]') #sauvegarder l'expression réguliere (r=expression reguliere)
    text = re.sub(regex, 'ا', str(text)) #remblacer des expressions par d'autres dans varia texte
    regex = re.compile(r'[ى]')
    text = re.sub(regex, 'ي', str(text))
    regex = re.compile(r'[ؤئ]')
    text = re.sub(regex, 'ء', str(text))
    text = re.sub('\n', ' ', str(text))
    text = re.sub(' و ', ' ', str(text))
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    text = text.rstrip()
#supprimer les diacritiques
    arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)   #verbose: séparer visuellement les expre et ajouter des commentaires
    text = re.sub(arabic_diacritics, '', text)
    

#supprimer les ponctuations
    #liste des ponctuations arabe et anglais qu'on veut supprimer
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''    # utiliser ''' :multiples caractére de chaine
    english_punctuations = string.punctuation              #(!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
    punctuations_list = arabic_punctuations + english_punctuations
    #table de mappage
    translator = str.maketrans('','', punctuations_list) # le 1er elem a remplacer/le 2eme le remplacement,3eme=supprimer
    text= text.translate(translator) #chaine ou les caractere sont remplacé par d'autres dns table de mappage

#supprimer les chaines répétés
    text=re.sub(r'(.)\1+', r'\1', str(text)) 

#supprimer les mots vides
    stopwords=nltk.corpus.stopwords.words('arabic') #les mots vides en arabe
    txt=[word for word in text if word not in stopwords]

#supprimer les émojis
    myre = re.compile(u'['
    u'\U0001F300-\U0001F64F'
    u'\U0001F680-\U0001F6FF'     
    u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    text=myre.sub('', text)
    
    
    return text

In [ ]:
filename='new_db.xlsx'

In [ ]:
#Importer le fichier excel
data=pd.read_excel(filename) #filename:le chemin d'acces au fichier excel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Cleaning DB
data['TWEET'] = data['TWEET'].apply(process)
data['TWEET'] = data['TWEET'].apply(nfx.remove_hashtags)
data['TWEET'] = data['TWEET'].apply(nfx.remove_userhandles)

In [ ]:
index=[]
for i in range(len(data)):
  if data.iloc[i].TWEET=='':
    index.append(i)

for i in index:
  data=data.drop(i)
data = data.reset_index().drop('index',axis=1)

In [ ]:
#Affichage des premiers elts
data.head()

,TWEET,LABEL
0,الاوليمبياد الجايه هكون لسه ف الكليه,0
1,عجز الموازنه وصل ل937 من الناتج المحلي يعني لس...,1
2,كتنا نيله ف حظنا الهباب,2
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,3
4,الاوليمبياد نظامها مختلف ومواعيد المونديال مكا...,0


In [ ]:
#Affichage des derniers elts
data.tail()

,TWEET,LABEL
29947,اتمني تعملو بطاقات خاصة او ذهبية لعملاء الداءم...,3
29948,المشويات عندهم اطيب مشويات اكلتها بحياتي والاس...,3
29949,المطعم متاز لغايه من ناحيه تجهيزات المطعم وايض...,3
29950,جينا عندهم الساعة 10 باليل مافي احد اتبرع يشوف...,1
29951,اذا كان الي بالشارع التجاري اوه ماشاء اله واله...,3


In [ ]:
#Remplacer chaque label par un entier
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

data['LABEL'] = data['LABEL'].apply(lambda x: encode_cat(x))

In [ ]:
#Split data into random train(80%) and test(20%) subsets
text_train, text_test, label_train, label_test = train_test_split(data.TWEET,data.LABEL,test_size=0.20, random_state=42)
print(text_train.shape,text_test.shape,label_train.shape,label_test.shape)  

(23961,) (5991,) (23961,) (5991,)


# WORD2VEC

# Preparation du model

In [ ]:
import spacy

In [ ]:
# load AraVec Spacy model
%cd /content/drive/MyDrive/Word2Vec
nlp = spacy.load("spacy.aravec.model")
%cd ..
%cd ..
%cd ..

/content/drive/MyDrive/Word2Vec
/content/drive/My Drive
/content/drive
/content


In [ ]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = process(text)
        return self.tokenizer(preprocessed)

In [ ]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

In [ ]:
#retourne le vecteur correspondant a la phrase x
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

# Machine Learning

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Get the sentence embeddings for the train dataset
vec_text_train = text_train.apply(lambda x: get_vec(x))
vec_text_train = vec_text_train.to_numpy()
vec_text_train = vec_text_train.reshape(-1, 1)
vec_text_train = np.concatenate(np.concatenate(vec_text_train, axis = 0), axis = 0).reshape(-1, 300)

# Get the sentence embeddings for the test dataset
vec_text_test = text_test.apply(lambda x: get_vec(x))
vec_text_test = vec_text_test.to_numpy()
vec_text_test = vec_text_test.reshape(-1, 1)
vec_text_test = np.concatenate(np.concatenate(vec_text_test, axis = 0), axis = 0).reshape(-1, 300)

In [ ]:
# create the model, train it, print scores
clf = LogisticRegression(solver='liblinear')
clf.fit(vec_text_train, label_train)
tag_pred = clf.predict(vec_text_test)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.60      0.66      0.63       314
           1       0.64      0.66      0.65      1184
           2       0.64      0.44      0.52       701
           3       0.73      0.90      0.80      2434
           4       0.59      0.29      0.39       470
           5       0.70      0.52      0.59       381
           6       0.78      0.71      0.75       231
           7       0.90      0.82      0.85       276

    accuracy                           0.70      5991
   macro avg       0.70      0.62      0.65      5991
weighted avg       0.69      0.70      0.69      5991



In [ ]:
# create the model, train it, print scores
SVM=svm.SVC()
SVM.fit(vec_text_train, label_train)
tag_pred = SVM.predict(vec_text_test)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.60      0.78      0.67       314
           1       0.65      0.74      0.69      1184
           2       0.64      0.48      0.55       701
           3       0.77      0.89      0.82      2434
           4       0.68      0.31      0.42       470
           5       0.76      0.52      0.62       381
           6       0.84      0.74      0.78       231
           7       0.97      0.75      0.85       276

    accuracy                           0.72      5991
   macro avg       0.74      0.65      0.68      5991
weighted avg       0.72      0.72      0.71      5991



In [ ]:
# create the model, train it, print scores
rf=RandomForestClassifier()
rf.fit(vec_text_train,label_train)
tag_pred = rf.predict(vec_text_test)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.53      0.63      0.58       314
           1       0.78      0.78      0.78      1184
           2       0.73      0.57      0.64       701
           3       0.80      0.92      0.85      2434
           4       0.75      0.49      0.60       470
           5       0.69      0.70      0.69       381
           6       0.82      0.63      0.71       231
           7       0.86      0.60      0.71       276

    accuracy                           0.76      5991
   macro avg       0.75      0.66      0.69      5991
weighted avg       0.76      0.76      0.76      5991



In [ ]:
# create the model, train it, print scores
nb=GaussianNB()
nb.fit(vec_text_train,label_train)
tag_pred = nb.predict(vec_text_test)
print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.42      0.71      0.53       314
           1       0.33      0.71      0.45      1184
           2       0.26      0.09      0.13       701
           3       0.82      0.32      0.47      2434
           4       0.23      0.14      0.17       470
           5       0.46      0.51      0.48       381
           6       0.33      0.68      0.45       231
           7       0.34      0.62      0.44       276

    accuracy                           0.42      5991
   macro avg       0.40      0.47      0.39      5991
weighted avg       0.53      0.42      0.40      5991



# Deep Learning

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report

In [ ]:
#create a word-to-index dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

In [ ]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 82

#Add 0 at the end of the list until it reaches the max length
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
#Create an embedding matrix where each row number will correspond to the index of the word in the corpus
embedding_matrix = np.zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = nlp(word).vector
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#--------------------------------------------------------------------------------------------------------------

In [ ]:
model1 = Sequential()
embedding_layer = Embedding(vocab_size, 300,  input_length=maxlen , trainable=False)
model1.add(embedding_layer)

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model1.add(GRU(256, activation='sigmoid',recurrent_activation='sigmoid' , return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model1.add(SimpleRNN(128))


model1.add(Dense(8, activation='softplus'))
model1.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 82, 300)           19754700  
_________________________________________________________________
gru_2 (GRU)                  (None, 82, 256)           428544    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 128)               49280     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
Total params: 20,233,556
Trainable params: 478,856
Non-trainable params: 19,754,700
_________________________________________________________________


In [ ]:
history = model1.fit(X_train, label_train, batch_size=128, epochs=20, verbose=1, validation_split=0.2)

score = model1.evaluate(X_test, label_test, verbose=1)

Epoch 1/20
150/150 [==============================] - 40s 248ms/step - loss: 1.7654 - acc: 0.4000 - val_loss: 1.6676 - val_acc: 0.4129
Epoch 2/20
150/150 [==============================] - 37s 249ms/step - loss: 1.6352 - acc: 0.4057 - val_loss: 1.4252 - val_acc: 0.4129
Epoch 3/20
150/150 [==============================] - 36s 242ms/step - loss: 1.3374 - acc: 0.4062 - val_loss: 1.1817 - val_acc: 0.4267
Epoch 4/20
150/150 [==============================] - 38s 252ms/step - loss: 1.1300 - acc: 0.4193 - val_loss: 1.1058 - val_acc: 0.4308
Epoch 5/20
150/150 [==============================] - 35s 236ms/step - loss: 1.0444 - acc: 0.4334 - val_loss: 1.0389 - val_acc: 0.4338
Epoch 6/20
150/150 [==============================] - 37s 247ms/step - loss: 0.9683 - acc: 0.4686 - val_loss: 1.0042 - val_acc: 0.4738
Epoch 7/20
150/150 [==============================] - 36s 237ms/step - loss: 0.9072 - acc: 0.4978 - val_loss: 0.9937 - val_acc: 0.5093
Epoch 8/20
150/150 [==============================] - 3

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.7296215891838074
Test Accuracy: 0.7986980676651001


In [ ]:
pred = np.argmax(model1.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.62      0.83      0.71       314
           1       0.73      0.85      0.79      1184
           2       0.82      0.58      0.68       701
           3       0.86      0.90      0.88      2434
           4       0.70      0.53      0.60       470
           5       0.80      0.64      0.71       381
           6       0.81      0.80      0.81       231
           7       0.96      0.86      0.90       276

    accuracy                           0.80      5991
   macro avg       0.79      0.75      0.76      5991
weighted avg       0.80      0.80      0.79      5991



In [ ]:
#----------------------------------------------------------------------------------

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model2.add(embedding_layer)

model2.add(LSTM(1024))

model2.add(Dense(8, activation='softmax'))
model2.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model2.fit(X_train, label_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

score = model2.evaluate(X_test, label_test, verbose=1)

Epoch 1/100
150/150 [==============================] - 20s 115ms/step - loss: 1.6734 - acc: 0.4242 - val_loss: 1.4021 - val_acc: 0.4849
Epoch 2/100
150/150 [==============================] - 17s 116ms/step - loss: 1.3194 - acc: 0.5042 - val_loss: 1.2489 - val_acc: 0.5523
Epoch 3/100
150/150 [==============================] - 18s 119ms/step - loss: 1.1552 - acc: 0.5934 - val_loss: 1.0764 - val_acc: 0.6305
Epoch 4/100
150/150 [==============================] - 18s 121ms/step - loss: 1.0560 - acc: 0.6347 - val_loss: 1.0621 - val_acc: 0.6439
Epoch 5/100
150/150 [==============================] - 19s 124ms/step - loss: 0.9923 - acc: 0.6678 - val_loss: 1.0570 - val_acc: 0.6311
Epoch 6/100
150/150 [==============================] - 20s 134ms/step - loss: 0.9437 - acc: 0.6706 - val_loss: 0.9824 - val_acc: 0.6658
Epoch 7/100
150/150 [==============================] - 19s 127ms/step - loss: 0.8804 - acc: 0.6985 - val_loss: 0.9341 - val_acc: 0.6829
Epoch 8/100
150/150 [===========================

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 1.2273974418640137
Test Accuracy: 0.7958604693412781


In [ ]:
pred = np.argmax(model2.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67       314
           1       0.82      0.74      0.78      1184
           2       0.76      0.66      0.70       701
           3       0.88      0.89      0.88      2434
           4       0.55      0.68      0.61       470
           5       0.69      0.76      0.72       381
           6       0.78      0.84      0.81       231
           7       0.92      0.87      0.89       276

    accuracy                           0.80      5991
   macro avg       0.75      0.77      0.76      5991
weighted avg       0.80      0.80      0.80      5991



In [ ]:
#------------------------------------------------------

In [ ]:
model3 = Sequential()

embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model3.add(embedding_layer)


model3.add(Conv1D(1024, 1, activation='softsign'))
model3.add(Conv1D(1024, 1, activation='softsign'))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(8, activation='softmax'))
model3.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model3.fit(X_train, label_train, batch_size=128, epochs=7, verbose=1, validation_split=0.2)

score = model3.evaluate(X_test, label_test, verbose=1)

Epoch 1/7
150/150 [==============================] - 7s 42ms/step - loss: 1.1826 - acc: 0.5948 - val_loss: 0.6708 - val_acc: 0.7657
Epoch 2/7
150/150 [==============================] - 6s 39ms/step - loss: 0.5782 - acc: 0.8075 - val_loss: 0.5869 - val_acc: 0.7937
Epoch 3/7
150/150 [==============================] - 6s 40ms/step - loss: 0.4523 - acc: 0.8595 - val_loss: 0.5516 - val_acc: 0.8103
Epoch 4/7
150/150 [==============================] - 6s 40ms/step - loss: 0.3658 - acc: 0.8941 - val_loss: 0.5296 - val_acc: 0.8310
Epoch 5/7
150/150 [==============================] - 6s 41ms/step - loss: 0.2903 - acc: 0.9237 - val_loss: 0.5218 - val_acc: 0.8337
Epoch 6/7
150/150 [==============================] - 6s 40ms/step - loss: 0.2347 - acc: 0.9416 - val_loss: 0.5171 - val_acc: 0.8323
Epoch 7/7
188/188 [==============================] - 1s 6ms/step - loss: 0.5655 - acc: 0.8373


In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.5654865503311157
Test Accuracy: 0.8372558951377869


In [ ]:
pred = np.argmax(model3.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.66      0.78      0.72       314
           1       0.87      0.81      0.84      1184
           2       0.77      0.73      0.75       701
           3       0.88      0.94      0.91      2434
           4       0.74      0.62      0.67       470
           5       0.76      0.76      0.76       381
           6       0.85      0.83      0.84       231
           7       0.95      0.90      0.92       276

    accuracy                           0.84      5991
   macro avg       0.81      0.80      0.80      5991
weighted avg       0.84      0.84      0.84      5991



In [ ]:
#--------------------------------------------------------------------------------------------